# 9. Agent

The key difference between an agent and a pipeline lies in their flexibility and decision-making capabilities.

Using “Agents” in frameworks like LangChain often introduces significant complexity. While the tool simplifies foundational tasks like API calls, stream handling, and vector database integration, the abstraction becomes overwhelming with agents.

These entities aim to manage dynamic workflows by chaining multiple tasks, such as calling APIs, interacting with tools, or solving problems iteratively. However, this flexibility comes at a cost: increased dependencies, debugging difficulties, and convoluted configurations. 

Developers often find themselves navigating layers of abstractions that obscure the underlying logic, making it harder to maintain, optimize, or understand the system, especially for straightforward tasks.

## Pipeline:

- A pipeline is a fixed sequence of steps that always processes input in the same way.
- Each step in a pipeline transforms the data and passes it to the next step.
- Example: If you want to preprocess input, pass it to an LLM, and then clean the output, a pipeline will always follow this sequence, regardless of the task.

Think of a pipeline as a well-defined assembly line where every step is predefined and linear.

## Agent:
- An agent is dynamic and can make decisions during execution.
- It analyzes the input and decides what tools or actions to use based on the task.
- Agents can use multiple tools or external resources (e.g., calculators, search engines) and switch between them as needed.
- Example: If the user asks a question that involves math, the agent might use a calculator tool; if the user asks for a definition, it might query an LLM instead.

Think of an agent as a problem solver that chooses the right tools for the job based on the input.

# Pipeline example

In [ ]:
from tools import llm # containing my ChatOllama
from langchain.chains import TransformChain

# Step 1: Preprocess input
def preprocess(inputs):
    return {"input": f"You are a helpful assistant. {inputs['input']}"}

# Step 2: Pass to LLM
def run_llm(inputs):
    response = llm.invoke([{"role": "user", "content": inputs["input"]}])
    return {"output": response.content.strip()}

# Step 3: Clean output
def postprocess(inputs):
    return {"final_output": inputs["output"]}

# Combine steps into a pipeline
preprocess_chain = TransformChain(input_variables=["input"], output_variables=["input"], transform=preprocess)
llm_chain = TransformChain(input_variables=["input"], output_variables=["output"], transform=run_llm)
postprocess_chain = TransformChain(input_variables=["output"], output_variables=["final_output"], transform=postprocess)
pipeline = preprocess_chain | llm_chain | postprocess_chain

# Test the pipeline
result = pipeline.invoke({"input": "What is 2 + 2?"})
print(result["final_output"])

# Agent example

In [8]:
from langchain.agents import initialize_agent, Tool

# Define tools
def translate_tool(text):
    print("Executing translate_tool")
    return "Bonjour" if "Hello" in text else "Unknown"

def calculator_tool(expression):
    print("Executing calculator_tool")
    try:
        return str(eval(expression))
    except Exception:
        return "Error"

# Initialize tools
tools = [
    Tool(name="Translator", func=translate_tool, description="Translate English to French."),
    Tool(name="Calculator", func=calculator_tool, description="Perform math calculations.")
]

# Initialize agent
agent = initialize_agent(tools=tools, llm=llm, agent="zero-shot-react-description")

# Test the agent with invoke
print(agent.invoke("Translate 'Hello' into French."))
print(agent.invoke("What is 2 + 2?"))

Executing translate_tool
{'input': "Translate 'Hello' into French.", 'output': 'Bonjour'}
Executing calculator_tool
Executing calculator_tool
Executing calculator_tool
Executing calculator_tool
Executing translate_tool
Executing translate_tool
Executing translate_tool
Executing translate_tool
Executing calculator_tool
Executing translate_tool
Executing calculator_tool
Executing translate_tool
Executing translate_tool
Executing translate_tool
Executing calculator_tool
{'input': 'What is 2 + 2?', 'output': 'Agent stopped due to iteration limit or time limit.'}


The main benefits of using an agent (initialize_agent) over a pipeline are flexibility, decision-making, and tool selection. Here’s how the agent in the given example compares to a pipeline:

## Benefits of the Agent:

### Dynamic Tool Selection:

- The agent analyzes the input and decides which tool (e.g., Translator or Calculator) to use.
- Example: For “Translate ‘Hello’ into French,” the agent uses the translate_tool, while for “What is 2 + 2?”, it uses the calculator_tool.
- In contrast, a pipeline always follows the same steps in a fixed order, even if some steps are unnecessary.

### Efficiency:

- The agent avoids invoking the LLM unnecessarily. 
- For math operations, it uses the faster and more efficient calculator_tool instead of processing the task with the LLM.
- A pipeline might waste resources by relying on the LLM for all tasks, regardless of the actual need.

### Scalability:

- An agent can handle a wide variety of tasks by adding more tools without requiring you to rewrite the logic.
- A pipeline would need to be restructured if new steps or different logic were required for additional tasks.

### Context-Aware Decision-Making:

- The agent uses a reasoning framework (like “zero-shot-react-description”) to interpret the user query and determine the best course of action.
- Pipelines do not interpret or make decisions; they simply process inputs linearly.

### Simpler Logic for Complex Workflows:

- Agents simplify workflows that require branching logic or tool selection.
- Without an agent, you’d have to write custom branching logic (e.g., if statements) to decide which tool or step to execute in a pipeline.
